In [1]:
import cv2
import os
import tensorflow as tf
from tensorflow import keras
from PIL import Image
import numpy as np
from sklearn.model_selection import train_test_split
from keras.utils import normalize
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.utils import to_categorical

In [6]:
image_directory='datasets/'

no_tumor_images=os.listdir(image_directory+ 'no/')
yes_tumor_images=os.listdir(image_directory+ 'yes/')
dataset=[]
label=[]

INPUT_SIZE=64
# print(no_tumor_images)

# path='no0.jpg'

# print(path.split('.')[1])

for i , image_name in enumerate(no_tumor_images):
    if(image_name.split('.')[1]=='jpg'):
        image=cv2.imread(image_directory+'no/'+image_name)
        image=Image.fromarray(image,'RGB')
        image=image.resize((INPUT_SIZE,INPUT_SIZE))
        dataset.append(np.array(image))
        label.append(0)

for i , image_name in enumerate(yes_tumor_images):
    if(image_name.split('.')[1]=='jpg'):
        image=cv2.imread(image_directory+'yes/'+image_name)
        image=Image.fromarray(image, 'RGB')
        image=image.resize((INPUT_SIZE,INPUT_SIZE))
        dataset.append(np.array(image))
        label.append(1)

dataset=np.array(dataset)
label=np.array(label)

In [7]:

x_train, x_test, y_train, y_test=train_test_split(dataset, label, test_size=0.2, random_state=0)

# Reshape = (n, image_width, image_height, n_channel)

# print(x_train.shape)
# print(y_train.shape)

# print(x_test.shape)
# print(y_test.shape)

x_train=normalize(x_train, axis=1)
x_test=normalize(x_test, axis=1)

y_train=to_categorical(y_train , num_classes=2)
y_test=to_categorical(y_test , num_classes=2)



In [8]:
# Model Building
# 64,64,3

model=Sequential()

model.add(Conv2D(32, (3,3), input_shape=(INPUT_SIZE, INPUT_SIZE, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(32, (3,3), kernel_initializer='he_uniform'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))


model.add(Conv2D(64, (3,3), kernel_initializer='he_uniform'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(2))
model.add(Activation('softmax'))

In [5]:
# Binary CrossEntropy= 1, sigmoid
# Categorical Cross Entryopy= 2 , softmax

model.compile(loss='categorical_crossentropy',optimizer='adam', metrics=['accuracy'])


model.fit(x_train, y_train, 
batch_size=16, 
verbose=1, epochs=10, 
validation_data=(x_test, y_test),
shuffle=False)


model.save('BrainTumor10EpochsCategorical.h5')

Epoch 1/10
150/150 [==============================] - 8s 45ms/step - loss: 0.5664 - accuracy: 0.7146 - val_loss: 0.4836 - val_accuracy: 0.7533
Epoch 2/10
150/150 [==============================] - 11s 74ms/step - loss: 0.4133 - accuracy: 0.8221 - val_loss: 0.3679 - val_accuracy: 0.8267
Epoch 3/10
150/150 [==============================] - 8s 52ms/step - loss: 0.3172 - accuracy: 0.8658 - val_loss: 0.2796 - val_accuracy: 0.8850
Epoch 4/10
150/150 [==============================] - 13s 84ms/step - loss: 0.2597 - accuracy: 0.9008 - val_loss: 0.2230 - val_accuracy: 0.9067
Epoch 5/10
150/150 [==============================] - 15s 100ms/step - loss: 0.1854 - accuracy: 0.9296 - val_loss: 0.1820 - val_accuracy: 0.9400
Epoch 6/10
150/150 [==============================] - 15s 100ms/step - loss: 0.1264 - accuracy: 0.9554 - val_loss: 0.1466 - val_accuracy: 0.9583
Epoch 7/10
150/150 [==============================] - 12s 83ms/step - loss: 0.0907 - accuracy: 0.9729 - val_loss: 0.1254 - val_accuracy:

In [9]:
import cv2
from keras.models import load_model
from PIL import Image
import numpy as np

In [13]:
model=load_model('BrainTumor10EpochsCategorical.h5')

image = cv2.imread('pred\pred0.jpg')

img=Image.fromarray(image)

img=img.resize((64,64))

img=np.array(img)

input_img=np.expand_dims(img, axis=0)

result=model.predict(input_img)
print(result)

1/1 [==============================] - 0s 123ms/step
[[1. 0.]]
